# Deep Neural Network (3)

* mnist
* initializer
* dropout
* batch norm (New)

* [강의출처](https://www.youtube.com/watch?v=HCEr5f-LfVE&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=17)
* [코드출처](https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-09_6_mnist_batchnorm.ipynb)

## Summary

* batch norm을 진행할 때, train 해줬을 때의 배치사이즈로 test 배치 사이즈를 만들어 예측해야지 잘 예측됨.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(42)
if device == 'cuda':
    torch.cuda.manual_seed_all(42)

## Dataset

In [3]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

## Constants

In [4]:
# parameters
nb_epochs = 10
batch_size = 32
learning_rate = 0.01
dims = 28 * 28
labels = 10
drop_rate = 0

In [5]:
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

In [6]:
# model
layer1 = nn.Linear(dims, 32, bias=True)
layer2 = nn.Linear(32, 32, bias=True)
layer3 = nn.Linear(32, 32, bias=True)
layer4 = nn.Linear(32, labels, bias=True)
activation = nn.ReLU()
dropout = nn.Dropout(p=drop_rate)
batch_norm1 = nn.BatchNorm1d(32)
batch_norm2 = nn.BatchNorm1d(32)
batch_norm3 = nn.BatchNorm1d(32)


model = nn.Sequential(layer1, batch_norm1, activation, dropout,
                      layer2, batch_norm2, activation, dropout,
                      layer3, batch_norm3, activation, dropout,
                      layer4
                     ).to(device)

In [7]:
# initializer
def init_weights(layer):
    if isinstance(layer, nn.Linear):
        nn.init.xavier_normal_(layer.weight)

In [8]:
model.apply(init_weights)

Sequential(
  (0): Linear(in_features=784, out_features=32, bias=True)
  (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0)
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Dropout(p=0)
  (8): Linear(in_features=32, out_features=32, bias=True)
  (9): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): Dropout(p=0)
  (12): Linear(in_features=32, out_features=10, bias=True)
)

In [9]:
# optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
for epoch in range(nb_epochs):
    # train mode - dropout, batch_norm
    model.train()

    avg_cost = 0
    train_batch = len(train_loader)

    for X, Y in train_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        hypothesis = model(X)
        cost = F.cross_entropy(hypothesis, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / train_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
    with torch.no_grad():
        # evaluation mode
        model.eval()
        
        acc = 0
        test_batch = len(test_loader)
        
        # cause of batch normalization
        for i, (X, Y) in enumerate(test_loader):
            X = X.view(-1, 28 * 28).to(device)
            Y = Y.to(device)

            prediction = model(X)
            correct_prediction = torch.argmax(prediction, 1) == Y
            acc += correct_prediction.float().mean() / test_batch
        
        print("TEST ACC: ", acc.item(), '\n')

print('Learning finished')

Epoch: 0001 cost = 0.310854107
TEST ACC:  0.9526240229606628 

Epoch: 0002 cost = 0.195059836
TEST ACC:  0.9627398252487183 

Epoch: 0003 cost = 0.163778320
TEST ACC:  0.9619386792182922 

Epoch: 0004 cost = 0.147405013
TEST ACC:  0.9651436805725098 

Epoch: 0005 cost = 0.133050516
TEST ACC:  0.9698508977890015 

Epoch: 0006 cost = 0.125474289
TEST ACC:  0.9711530208587646 

Epoch: 0007 cost = 0.120682582
TEST ACC:  0.9719541668891907 

Epoch: 0008 cost = 0.112345397
TEST ACC:  0.9717543721199036 

Epoch: 0009 cost = 0.107398607
TEST ACC:  0.9713537096977234 

Epoch: 0010 cost = 0.102540843
TEST ACC:  0.9730561971664429 

Learning finished
